In [ ]:
def plot_pmf_fit_grid(fig, ax, animal_names, paths_all_animals, min_stims=5, stims_equal=False, show_plot=False):
    import matplotlib.cm as cm

    session_info = []
    for animal in animal_names:
        session_files = paths_all_animals[animal]
        for session_file in session_files:
            session_data = pd.read_hdf(session_file)
            sel = session_data[session_data.response_side.isin([0,1])] # select only trials where the subject responded
            responded_right = np.array(sel.response_side == 1).astype(int) # select the response = 1 (i.e. the left side) and cast to integer datatype (number)   
            sel_stim_rates = sel.stimulus_event_timestamps.str.len()
            unique_stim_rates = len(np.unique(sel_stim_rates))
            if (stims_equal and unique_stim_rates == min_stims) or (not stims_equal and unique_stim_rates >= min_stims):
                session_info.append((sel_stim_rates, responded_right))

    count = len(session_info)
    colors = cm.plasma(np.linspace(0, 1, count)) # get a sequential colormap
    xticks = dict({2:[4,20], 4:[4,8,16,20], 5:[4,8,12,16,20], 6:[4,5,6,18,19,20], 9:[4,6,8,10,12,14,16,18,20]})
    
    for color_count, (sel_stim_rates, responded_right) in enumerate(session_info):
        try:
            res = fit_psychometric(sel_stim_rates.astype(float),responded_right.astype(float), min_required_stim_values=min_stims)

            # now plot
            for s,c in zip(res['stims'],res['p_side_ci']):
                ax.plot(s*np.array([1,1]),c,color=colors[color_count], linestyle='-', linewidth=2)
            ax.plot(res['stims'],res['p_side'],'o',markerfacecolor = 'lightgray',markersize = 6, color=colors[color_count])

            nx = np.linspace(res['stims'].min(),res['stims'].max(),100)
            ax.plot(nx,res['function'](*res['fit_params'],nx),color=colors[color_count], linewidth=2)
            ax.set_ylabel('P$_{right}$',fontsize = 18)  # set the y-axis label with latex nomenclature
            ax.set_xlabel('Stimulus rate (Hz)', fontsize = 14); # set the x-axis label
            ax.set_xticks(xticks[min_stims])
        except Exception as e:
            continue

    # add colorbar
    sm = plt.cm.ScalarMappable(cmap=cm.plasma, norm=plt.Normalize(vmin=0, vmax=count-1))
    sm._A = []
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label('Sessions', rotation=270, labelpad=20)

    if show_plot:
        plt.show()
    
    return None

def plot_PMF_animals(animal_names, paths_all_animals, minstims, stimsequal):
    # Calculate the number of rows and columns for the subplot grid
    n = len(animal_names)
    ncols = int(np.ceil(np.sqrt(n)))
    nrows = int(np.ceil(n / ncols))

    fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))

    # Flatten the axes array and remove extra subplots
    axs = axs.flatten()
    for ax in axs[n:]:
        fig.delaxes(ax)

    for i, ax in enumerate(axs[:n]):
        plot_pmf_fit_grid(fig, ax, [animal_names[i]], paths_all_animals, min_stims=minstims, stims_equal=stimsequal)
        ax.set_title(animal_names[i])

    fig.tight_layout()

    plt.show()

# plt.close()
first_cohort = ['GRB001', 'GRB002', 'GRB003', 'GRB004']
second_cohort = ['GRB005', 'GRB006', 'GRB007']
third_cohort = ['GRB015', 'GRB016', 'GRB017', 'GRB018']

plot_PMF_animals(animal_names=second_cohort, paths_all_animals=paths_all_animals, minstims=, stimsequal=False)
# plt.savefig("/Users/gabriel/lib/lab-projects/chipmunk_behavior/figures/firstcohortgrid.pdf", format='pdf')

## PMF parameter plotting (backlog)

In [ ]:
def get_pmf_parameters(file_names):
    # file_names = chiCa.pick_files_multi_session("chipmunk", "*.h5")
    bias = []
    sensitivity = []
    left_lapse = []
    right_lapse = []
    for file in file_names:
        session_data = pd.read_hdf(file)
        sel = session_data[session_data.response_side.isin([0,1])] # select only trials where the subject responded
        responded_right = npx.array(sel.response_side == 1).astype(int) # select the response = 1 (i.e. the left side) and cast to integer datatype (number)   
        sel_stim_rates = np.array([len(timestamps) for timestamps in sel.stimulus_event_timestamps])


        res = fit_psychometric(sel_stim_rates.astype(float),responded_right.astype(float), min_required_stim_values=5)

        try:
            bias.append(res['fit_params'][0])
            sensitivity.append(res['fit_params'][1])
            left_lapse.append(res['fit_params'][2])
            right_lapse.append(res['fit_params'][3])
        except:
            bias.append(np.nan)
            sensitivity.append(np.nan)
            left_lapse.append(np.nan)
            right_lapse.append(np.nan)

    return bias, sensitivity, left_lapse, right_lapse

GRB001_pmf_parameters = get_pmf_parameters(file_names)
GRB002_pmf_parameters = get_pmf_parameters(file_names)
GRB003_pmf_parameters = get_pmf_parameters(file_names)
GRB004_pmf_parameters = get_pmf_parameters(file_names)

GRB001_params = dict({'bias':GRB001_pmf_parameters[0], 'sensitivity':GRB001_pmf_parameters[1], 'left_lapse':GRB001_pmf_parameters[2], 'right_lapse':GRB001_pmf_parameters[3]})
GRB002_params = dict({'bias':GRB002_pmf_parameters[0], 'sensitivity':GRB002_pmf_parameters[1], 'left_lapse':GRB002_pmf_parameters[2], 'right_lapse':GRB002_pmf_parameters[3]})
GRB003_params = dict({'bias':GRB003_pmf_parameters[0], 'sensitivity':GRB003_pmf_parameters[1], 'left_lapse':GRB003_pmf_parameters[2], 'right_lapse':GRB003_pmf_parameters[3]})
GRB004_params = dict({'bias':GRB004_pmf_parameters[0], 'sensitivity':GRB004_pmf_parameters[1], 'left_lapse':GRB004_pmf_parameters[2], 'right_lapse':GRB004_pmf_parameters[3]})

### Parameters lineplot

In [ ]:
plt.close()

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

max_length = max(len(bias_GRB001), len(bias_GRB002), len(bias_GRB003), len(bias_GRB004))

bias_GRB001 = np.pad(bias_GRB001, (0, max_length - len(bias_GRB001)), mode='constant', constant_values=np.nan)
bias_GRB002 = np.pad(bias_GRB002, (0, max_length - len(bias_GRB002)), mode='constant', constant_values=np.nan)
bias_GRB003 = np.pad(bias_GRB003, (0, max_length - len(bias_GRB003)), mode='constant', constant_values=np.nan)
bias_GRB004 = np.pad(bias_GRB004, (0, max_length - len(bias_GRB004)), mode='constant', constant_values=np.nan)

bias = np.array([bias_GRB001, bias_GRB002, bias_GRB003, bias_GRB004])
labels = ['GRB001', 'GRB002', 'GRB003', 'GRB004']

axs[0, 0].plot(bias.T, label=labels)
axs[0, 0].set_title('Bias')
axs[0, 0].legend()

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

max_length = max(len(sensitivity_GRB001), len(sensitivity_GRB002), len(sensitivity_GRB003), len(sensitivity_GRB004))

sensitivity_GRB001 = np.pad(sensitivity_GRB001, (0, max_length - len(sensitivity_GRB001)), mode='constant', constant_values=np.nan)
sensitivity_GRB002 = np.pad(sensitivity_GRB002, (0, max_length - len(sensitivity_GRB002)), mode='constant', constant_values=np.nan)
sensitivity_GRB003 = np.pad(sensitivity_GRB003, (0, max_length - len(sensitivity_GRB003)), mode='constant', constant_values=np.nan)
sensitivity_GRB004 = np.pad(sensitivity_GRB004, (0, max_length - len(sensitivity_GRB004)), mode='constant', constant_values=np.nan)

sensitivity = np.array([sensitivity_GRB001, sensitivity_GRB002, sensitivity_GRB003, sensitivity_GRB004])
labels = ['GRB001', 'GRB002', 'GRB003', 'GRB004']

axs[0, 1].plot(sensitivity.T, label=labels)
axs[0, 1].set_title('Sensitivity')
axs[0, 1].legend()

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

max_length = max(len(left_lapse_GRB001), len(left_lapse_GRB002), len(left_lapse_GRB003), len(left_lapse_GRB004))

left_lapse_GRB001 = np.pad(left_lapse_GRB001, (0, max_length - len(left_lapse_GRB001)), mode='constant', constant_values=np.nan)
left_lapse_GRB002 = np.pad(left_lapse_GRB002, (0, max_length - len(left_lapse_GRB002)), mode='constant', constant_values=np.nan)
left_lapse_GRB003 = np.pad(left_lapse_GRB003, (0, max_length - len(left_lapse_GRB003)), mode='constant', constant_values=np.nan)
left_lapse_GRB004 = np.pad(left_lapse_GRB004, (0, max_length - len(left_lapse_GRB004)), mode='constant', constant_values=np.nan)

left_lapse = np.array([left_lapse_GRB001, left_lapse_GRB002, left_lapse_GRB003, left_lapse_GRB004])
labels = ['GRB001', 'GRB002', 'GRB003', 'GRB004']

axs[1, 0].plot(left_lapse.T, label=labels)
axs[1, 0].set_title('Left Lapse')
axs[1, 0].legend()

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

max_length = max(len(right_lapse_GRB001), len(right_lapse_GRB002), len(right_lapse_GRB003), len(right_lapse_GRB004))

right_lapse_GRB001 = np.pad(right_lapse_GRB001, (0, max_length - len(right_lapse_GRB001)), mode='constant', constant_values=np.nan)
right_lapse_GRB002 = np.pad(right_lapse_GRB002, (0, max_length - len(right_lapse_GRB002)), mode='constant', constant_values=np.nan)
right_lapse_GRB003 = np.pad(right_lapse_GRB003, (0, max_length - len(right_lapse_GRB003)), mode='constant', constant_values=np.nan)
right_lapse_GRB004 = np.pad(right_lapse_GRB004, (0, max_length - len(right_lapse_GRB004)), mode='constant', constant_values=np.nan)

right_lapse = np.array([right_lapse_GRB001, right_lapse_GRB002, right_lapse_GRB003, right_lapse_GRB004])
labels = ['GRB001', 'GRB002', 'GRB003', 'GRB004']

axs[1, 1].plot(right_lapse.T, label=labels)
axs[1, 1].set_title('Right Lapse')
axs[1, 1].legend()

plt.show()

### Parameters boxplot with matplotlib

In [ ]:
import seaborn as sns

plt.close()

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

bias_data = [bias_GRB001[~np.isnan(bias_GRB001)],
             bias_GRB002[~np.isnan(bias_GRB002)],
             bias_GRB003[~np.isnan(bias_GRB003)],
             bias_GRB004[~np.isnan(bias_GRB004)]]

boxprops = dict(linestyle='-', linewidth=2)
boxprops['color'] = sns.color_palette('Blues')[2]
axs[0, 0].boxplot(bias_data, boxprops=boxprops)
axs[0, 0].set_title('Bias')
axs[0, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

sensitivity_data = [sensitivity_GRB001[~np.isnan(sensitivity_GRB001)],
                    sensitivity_GRB002[~np.isnan(sensitivity_GRB002)],
                    sensitivity_GRB003[~np.isnan(sensitivity_GRB003)],
                    sensitivity_GRB004[~np.isnan(sensitivity_GRB004)]]

boxprops = dict(linestyle='-', linewidth=2, color=sns.color_palette('Reds')[3])
axs[0, 1].boxplot(sensitivity_data, boxprops=boxprops)
axs[0, 1].set_title('Sensitivity')
axs[0, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

left_lapse_data = [left_lapse_GRB001[~np.isnan(left_lapse_GRB001)],
                   left_lapse_GRB002[~np.isnan(left_lapse_GRB002)],
                   left_lapse_GRB003[~np.isnan(left_lapse_GRB003)],
                   left_lapse_GRB004[~np.isnan(left_lapse_GRB004)]]

boxprops = dict(linestyle='-', linewidth=2, color=sns.color_palette('Greens')[3])
axs[1, 0].boxplot(left_lapse_data, boxprops=boxprops)
axs[1, 0].set_title('Left Lapse')
axs[1, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

right_lapse_data = [right_lapse_GRB001[~np.isnan(right_lapse_GRB001)],
                    right_lapse_GRB002[~np.isnan(right_lapse_GRB002)],
                    right_lapse_GRB003[~np.isnan(right_lapse_GRB003)],
                    right_lapse_GRB004[~np.isnan(right_lapse_GRB004)]]

boxprops = dict(linestyle='-', linewidth=2, color=sns.color_palette('Purples')[3])
axs[1, 1].boxplot(right_lapse_data, boxprops=boxprops)
axs[1, 1].set_title('Right Lapse')
axs[1, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

plt.show()

### Parameters boxplot with seaborn

In [ ]:
import seaborn as sns

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

bias_data = [bias_GRB001[~np.isnan(bias_GRB001)],
             bias_GRB002[~np.isnan(bias_GRB002)],
             bias_GRB003[~np.isnan(bias_GRB003)],
             bias_GRB004[~np.isnan(bias_GRB004)]]

sns.boxplot(ax=axs[0, 0], data=bias_data, color=sns.color_palette('Blues')[2])
sns.stripplot(ax=axs[0, 0], data=bias_data, color='grey')
axs[0, 0].set_title('Bias')
axs[0, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

sensitivity_data = [sensitivity_GRB001[~np.isnan(sensitivity_GRB001)],
                    sensitivity_GRB002[~np.isnan(sensitivity_GRB002)],
                    sensitivity_GRB003[~np.isnan(sensitivity_GRB003)],
                    sensitivity_GRB004[~np.isnan(sensitivity_GRB004)]]

sns.boxplot(ax=axs[0, 1], data=sensitivity_data, color=sns.color_palette('Reds')[3])
sns.stripplot(ax=axs[0, 1], data=sensitivity_data, color='grey')
axs[0, 1].set_title('Sensitivity')
axs[0, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

left_lapse_data = [left_lapse_GRB001[~np.isnan(left_lapse_GRB001)],
                   left_lapse_GRB002[~np.isnan(left_lapse_GRB002)],
                   left_lapse_GRB003[~np.isnan(left_lapse_GRB003)],
                   left_lapse_GRB004[~np.isnan(left_lapse_GRB004)]]

sns.boxplot(ax=axs[1, 0], data=left_lapse_data, color=sns.color_palette('Greens')[3])
sns.stripplot(ax=axs[1, 0], data=left_lapse_data, color='grey')
axs[1, 0].set_title('Left Lapse')
axs[1, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

right_lapse_data = [right_lapse_GRB001[~np.isnan(right_lapse_GRB001)],
                    right_lapse_GRB002[~np.isnan(right_lapse_GRB002)],
                    right_lapse_GRB003[~np.isnan(right_lapse_GRB003)],
                    right_lapse_GRB004[~np.isnan(right_lapse_GRB004)]]

sns.boxplot(ax=axs[1, 1], data=right_lapse_data, color=sns.color_palette('Purples')[3])
sns.stripplot(ax=axs[1, 1], data=right_lapse_data, color='grey')
axs[1, 1].set_title('Right Lapse')
axs[1, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

plt.show()

### Violinplot

In [ ]:
import seaborn as sns

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
colors = sns.color_palette("hls", 4)

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

bias_data = [bias_GRB001[~np.isnan(bias_GRB001)],
             bias_GRB002[~np.isnan(bias_GRB002)],
             bias_GRB003[~np.isnan(bias_GRB003)],
             bias_GRB004[~np.isnan(bias_GRB004)]]

sns.violinplot(ax=axs[0, 0], data=bias_data, inner='point', palette=colors)
axs[0, 0].set_title('Bias')
axs[0, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

sensitivity_data = [sensitivity_GRB001[~np.isnan(sensitivity_GRB001)],
                    sensitivity_GRB002[~np.isnan(sensitivity_GRB002)],
                    sensitivity_GRB003[~np.isnan(sensitivity_GRB003)],
                    sensitivity_GRB004[~np.isnan(sensitivity_GRB004)]]

sns.violinplot(ax=axs[0, 1], data=sensitivity_data, inner='point', palette=colors)
axs[0, 1].set_title('Sensitivity')
axs[0, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

left_lapse_data = [left_lapse_GRB001[~np.isnan(left_lapse_GRB001)],
                   left_lapse_GRB002[~np.isnan(left_lapse_GRB002)],
                   left_lapse_GRB003[~np.isnan(left_lapse_GRB003)],
                   left_lapse_GRB004[~np.isnan(left_lapse_GRB004)]]

sns.violinplot(ax=axs[1, 0], data=left_lapse_data, inner='point', palette=colors)
axs[1, 0].set_title('Left Lapse')
axs[1, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

right_lapse_data = [right_lapse_GRB001[~np.isnan(right_lapse_GRB001)],
                    right_lapse_GRB002[~np.isnan(right_lapse_GRB002)],
                    right_lapse_GRB003[~np.isnan(right_lapse_GRB003)],
                    right_lapse_GRB004[~np.isnan(right_lapse_GRB004)]]

sns.violinplot(ax=axs[1, 1], data=right_lapse_data, inner='point', palette=colors)
axs[1, 1].set_title('Right Lapse')
axs[1, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

plt.show()

### Rainplots with PMF parameters

In [ ]:
import seaborn as sns
import ptitprince as pt

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
colors = sns.color_palette("hls", 4)

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

bias_data = [bias_GRB001[~np.isnan(bias_GRB001)],
             bias_GRB002[~np.isnan(bias_GRB002)],
             bias_GRB003[~np.isnan(bias_GRB003)],
             bias_GRB004[~np.isnan(bias_GRB004)]]

pt.RainCloud(ax=axs[0, 0], data=bias_data,  palette=colors)

axs[0, 0].set_title('Bias')
axs[0, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

sensitivity_data = [sensitivity_GRB001[~np.isnan(sensitivity_GRB001)],
                    sensitivity_GRB002[~np.isnan(sensitivity_GRB002)],
                    sensitivity_GRB003[~np.isnan(sensitivity_GRB003)],
                    sensitivity_GRB004[~np.isnan(sensitivity_GRB004)]]

pt.RainCloud(ax=axs[0, 1], data=sensitivity_data, palette=colors)
axs[0, 1].set_title('Sensitivity')
axs[0, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

left_lapse_data = [left_lapse_GRB001[~np.isnan(left_lapse_GRB001)],
                   left_lapse_GRB002[~np.isnan(left_lapse_GRB002)],
                   left_lapse_GRB003[~np.isnan(left_lapse_GRB003)],
                   left_lapse_GRB004[~np.isnan(left_lapse_GRB004)]]

pt.RainCloud(ax=axs[1, 0], data=left_lapse_data, palette=colors)
axs[1, 0].set_title('Left Lapse')
axs[1, 0].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

right_lapse_data = [right_lapse_GRB001[~np.isnan(right_lapse_GRB001)],
                    right_lapse_GRB002[~np.isnan(right_lapse_GRB002)],
                    right_lapse_GRB003[~np.isnan(right_lapse_GRB003)],
                    right_lapse_GRB004[~np.isnan(right_lapse_GRB004)]]

pt.RainCloud(ax=axs[1, 1], data=right_lapse_data, palette=colors)
axs[1, 1].set_title('Right Lapse')
axs[1, 1].set_xticklabels(['GRB001', 'GRB002', 'GRB003', 'GRB004'])

plt.show()

### Kdeplot colored by sex

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
colors= ['r', 'lightcoral', 'b', 'deepskyblue']
labels = ['GRB001', 'GRB002', 'GRB003', 'GRB004']

# Plot bias
bias_GRB001 = np.array(GRB001_params['bias'])
bias_GRB002 = np.array(GRB002_params['bias'])
bias_GRB003 = np.array(GRB003_params['bias'])
bias_GRB004 = np.array(GRB004_params['bias'])

bias_data = [bias_GRB001[~np.isnan(bias_GRB001)],
             bias_GRB002[~np.isnan(bias_GRB002)],
             bias_GRB003[~np.isnan(bias_GRB003)],
             bias_GRB004[~np.isnan(bias_GRB004)]]

sns.kdeplot(ax=axs[0, 0], data=bias_data, palette=colors, fill=False)
# axs[0, 0].set_ylim([0,1.7])
axs[0, 0].set_title('Bias')
axs[0, 0].legend(labels)

# Plot sensitivity
sensitivity_GRB001 = np.array(GRB001_params['sensitivity'])
sensitivity_GRB002 = np.array(GRB002_params['sensitivity'])
sensitivity_GRB003 = np.array(GRB003_params['sensitivity'])
sensitivity_GRB004 = np.array(GRB004_params['sensitivity'])

sensitivity_data = [sensitivity_GRB001[~np.isnan(sensitivity_GRB001)],
                    sensitivity_GRB002[~np.isnan(sensitivity_GRB002)],
                    sensitivity_GRB003[~np.isnan(sensitivity_GRB003)],
                    sensitivity_GRB004[~np.isnan(sensitivity_GRB004)]]

sns.kdeplot(ax=axs[0, 1], data=sensitivity_data, palette=colors, fill=False)
# axs[0, 1].set_ylim([0,1.7])
axs[0, 1].set_title('Sensitivity')
axs[0, 1].legend(labels)

# Plot left lapse
left_lapse_GRB001 = np.array(GRB001_params['left_lapse'])
left_lapse_GRB002 = np.array(GRB002_params['left_lapse'])
left_lapse_GRB003 = np.array(GRB003_params['left_lapse'])
left_lapse_GRB004 = np.array(GRB004_params['left_lapse'])

left_lapse_data = [left_lapse_GRB001[~np.isnan(left_lapse_GRB001)],
                   left_lapse_GRB002[~np.isnan(left_lapse_GRB002)],
                   left_lapse_GRB003[~np.isnan(left_lapse_GRB003)],
                   left_lapse_GRB004[~np.isnan(left_lapse_GRB004)]]

sns.kdeplot(ax=axs[1, 0], data=left_lapse_data, palette=colors, fill=False)
# axs[1, 0].set_ylim([0,1.7])
axs[1, 0].set_title('Left Lapse')
axs[1, 0].legend(labels)

# Plot right lapse
right_lapse_GRB001 = np.array(GRB001_params['right_lapse'])
right_lapse_GRB002 = np.array(GRB002_params['right_lapse'])
right_lapse_GRB003 = np.array(GRB003_params['right_lapse'])
right_lapse_GRB004 = np.array(GRB004_params['right_lapse'])

right_lapse_data = [right_lapse_GRB001[~np.isnan(right_lapse_GRB001)],
                    right_lapse_GRB002[~np.isnan(right_lapse_GRB002)],
                    right_lapse_GRB003[~np.isnan(right_lapse_GRB003)],
                    right_lapse_GRB004[~np.isnan(right_lapse_GRB004)]]

sns.kdeplot(ax=axs[1, 1], data=right_lapse_data, palette=colors, fill=False)
# axs[1, 1].set_ylim([0,1.7])
axs[1, 1].set_title('Right Lapse')
axs[1, 1].legend(labels)

plt.show()